## Easy questions
1) Which attributes have null values and if they do how many null values are there?  
2) How many items from the metadata dataset do not fall under the 5-core constraint?  
3) What is the average price and sales rank (how well the product sells within its category) of those products?  
4) Which product category contains the most of those non-5-core products?

## Medium questions
1) Average rating score of all products and also the average by each category.  
2) Can a user review the same product twice?  
3) Can two products with different IDs share the same name?

## Hard questions
4) Does the length of the written review corresponds to the rating of the product?  
5) Whether individuals tend to write reviews within one product category or whether they tend to review a variety of products. Avg. number of categories reviewed per person.  
6) Do they give high ratings within one product category and low ratings in a different category. Avg. rating per category per person.  
7) Clusters of reviewers who review many of the same products.   
8) Average distance spanning any two reviewers or any two products.  

In [1]:
#setup neo4j instance connection
from py2neo import Graph
from py2neo import *
#enter url for neo4j server here
url=""
str_list=['http:/',url,'db','data']
#enter password here
pw=""

authenticate(url,"neo4j", pw)
graph = Graph('/'.join(str_list))


In [ ]:
##1) Which attributes have null values and if they do how many null values are there?

#number Persons with name = NULL; if returned count = 0 then property does not contain NULL
query="MATCH (p:Person) WHERE p.name IS NULL RETURN count(p) as name_null_cnt" #2194

#number of distinct properties combination each person node has
#since there are person nodes with only id attribute hence all other properties other than id have null values
#same goes for product
query2="MATCH (n:Person) RETURN DISTINCT keys(n)"
query3="MATCH (n:Product) RETURN DISTINCT keys(n)"
#find node with the most to least properties
query4="MATCH (n) RETURN labels(n), keys(n), size(keys(n)), count(*) ORDER BY size(keys(n)) DESC"

In [ ]:
###2) How many items from the metadata dataset do not fall under the 5-core constraint?
#find products with no reviews = 0
query5="MATCH (n:Product) WHERE not ((n)<-[:Reviewed]-(:Person)) RETURN count(n)"

################
#person who reviewed no products
#receives SERVICE UNAVAILABLE error, might be OPTIONAL MATCH call
"MATCH (a:Person) OPTIONAL MATCH (a)-[r:Reviewed]->() RETURN a.id, r"

In [ ]:
###3) What is the average price of those products?
#overall average price
query6="MATCH (n:Product) RETURN AVG(tointeger(n.price))"
#average price by category
query7="MATCH (n:Product) RETURN AVG(tointeger(n.price)),n.categories"

In [ ]:
###4) Which product category contains the most of those non-5-core products?
#get number of products per category
"MATCH (n:Product) RETURN count(n),n.categories"

In [ ]:
###1) Average rating score of all products and also the average by each category.
#overall average rating
query6="MATCH (n:Product)-[r]-() RETURN avg(tointeger(r.score))"
#average rating by category
query7="MATCH (n:Product)-[r]-() RETURN count(n),avg(tointeger(r.score)),n.categories"

In [ ]:
###2) Can a user review the same product twice?
#user cannot review same product twice
query8="MATCH (a)-[r]->(b) WITH a, b, TAIL (COLLECT (r)) as rr WHERE length(rr) > 0 RETURN a, b"

In [ ]:
###3) Can two products with different IDs share the same name?
#There are products with different ID that share the same name
query9="MATCH (n:Product) WITH n.name as name, count(*) as cnt WHERE cnt>1 RETURN name, cnt"

In [ ]:
#shortest path from A to B
#make sure direction of arrows are correct
#ex. (a)-[*..30]->(b) or (a)<-[*..30]-(b) returns zero
query10="""MATCH (a:Product {id: '0007278446'}), (b:Product {id: '0007280777'}) 
           MATCH p=shortestPath((a)<-[*..30]->(b)) 
           RETURN p"""

#longest shortest path between two persons
query11="""MATCH (a:Person), (b:Person),
           path = allShortestPaths((a)-[*..30]-(b))
           WHERE a<>b #prevents common nodes error
           RETURN path
           ORDER BY LENGTH(path) DESC
           LIMIT 1"""

## DEMO CODE

Difficulties:  
1) directly binding prefix, [r:Reviewed], to list of relationships is decomissioned.  
2) simple calculations, such as average and sum, become cumbersome; Need to EXTRACT from list first or use REDUCE function. ex. reduce(total = 0, n IN nodes(p)| total + n.score) AS cost  


In [ ]:
#Find a book that have the words "..."
query="""MATCH (a:Product)
WHERE a.categories="[['Books']]" and a.name=~"The Warriors.*"
RETURN a limit 5"""

#calculate cost of path between two nodes
query="""MATCH p=(a:Product {id: '0007278446'})<-[:Reviewed*2]->(b:Product {id: '0060080817'})
RETURN REDUCE (total = 0, r in relationships(p) | total + tointeger(r.score))"""

#get all 2 degree path for node, id = 0007278446; demo in neo4J to show graph
query="""MATCH p=(a:Product {id: '0007278446'})<-[:Reviewed*2]->(b:Product)
RETURN p"""

#Return top 5 highest cost path where second edge weight > first edge weight (Recommended Products)
query="""MATCH p=(a:Product {id: '0007278446'})<-[:Reviewed*2]->(b:Product)
WITH  extract(s in relationships(p) | tointeger(s.score)) as p_collect,a,b
WITH  p_collect[0] as s1,p_collect[1] as s2,a,b
WHERE s2>=s1 and (b.name is not null or b.name <>"")
RETURN a.id,b.id,b.name, s1,s2,s1+s2 as cost
ORDER BY cost DESC LIMIT 5"""

In [ ]:
graph.evaluate(query)